In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install --upgrade huggingface_hub sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 101.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transf

In [ ]:
# ✅ Clean install to fix version conflicts
!pip uninstall -y transformers sentence-transformers
!pip install -U sentence-transformers==2.2.2 transformers==4.31.0


Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
Found existing installation: sentence-transformers 2.2.2
Uninstalling sentence-transformers-2.2.2:
  Successfully uninstalled sentence-transformers-2.2.2
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)


In [ ]:
import os
os.kill(os.getpid(), 9)


In [ ]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
from tqdm import tqdm

# ✅ Paths
DATA_PATH = "/content/drive/MyDrive/complaint-insightbot/filtered_complaints.csv"
VECTOR_STORE_DIR = "/content/drive/MyDrive/complaint-insightbot/vector_store"
os.makedirs(VECTOR_STORE_DIR, exist_ok=True)

INDEX_PATH = os.path.join(VECTOR_STORE_DIR, "faiss_index")
METADATA_PATH = os.path.join(VECTOR_STORE_DIR, "metadata.pkl")

# ✅ Parameters
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
BATCH_SIZE = 128  # 👈 safer for T4 GPU, you can try 256 if you're sure
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

# ✅ Load data
print("📥 Loading data...")
df = pd.read_csv(DATA_PATH)

# ✅ Chunking
print("✂️ Chunking texts...")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
)

flattened = []
for idx, text in tqdm(enumerate(df['cleaned_narrative']), total=len(df), desc="🔨 Splitting"):
    try:
        chunks = splitter.split_text(text)
        complaint_id = df.iloc[idx].get('complaint_id', idx)
        product = df.iloc[idx].get('product_mapped', None)
        for i, chunk in enumerate(chunks):
            flattened.append({
                "chunk_id": f"{complaint_id}_{i}",
                "complaint_id": complaint_id,
                "product": product,
                "chunk_text": chunk
            })
    except Exception as e:
        print(f"⚠️ Failed to split text at row {idx}: {e}")

# ✅ Embedding
print("🔍 Embedding chunks...")
texts = [item["chunk_text"] for item in flattened]
model = SentenceTransformer(EMBEDDING_MODEL_NAME, device='cuda')

all_embeddings = []
for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="🚀 Encoding"):
    batch = texts[i:i + BATCH_SIZE]
    try:
        emb = model.encode(batch, convert_to_numpy=True)
        all_embeddings.append(emb)
    except Exception as e:
        print(f"❌ Failed to embed batch {i}-{i+BATCH_SIZE}: {e}")

embeddings = np.vstack(all_embeddings)

# ✅ FAISS Index
print("📦 Building FAISS index...")
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# ✅ Save
print("💾 Saving index and metadata...")
faiss.write_index(index, INDEX_PATH)
with open(METADATA_PATH, "wb") as f:
    pickle.dump(flattened, f)

print("✅ All done! Vector store is ready.")


📥 Loading data...
✂️ Chunking texts...


🔨 Splitting: 100%|██████████| 459583/459583 [01:54<00:00, 4006.29it/s]


🔍 Embedding chunks...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🚀 Encoding: 100%|██████████| 8947/8947 [26:38<00:00,  5.60it/s]


📦 Building FAISS index...
💾 Saving index and metadata...
✅ All done! Vector store is ready.


In [ ]:
!pip install -q faiss-cpu sentence-transformers tqdm